In [16]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler
from model import LSTMModel

# Dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, data, window_size):
        self.data = data
        self.window_size = window_size

    def __len__(self):
        return len(self.data) - self.window_size * 2  # 修正: データの範囲を正しく設定

    def __getitem__(self, idx):
        return (self.data[idx:idx+self.window_size], 
                self.data[idx+self.window_size:idx+self.window_size*2])

# Data preprocessing
df = pd.read_csv('data/weather_data_Nerima_5y.csv')
df['ave_temperature'] = df['ave_temperature'].interpolate()
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
df = df[['date', 'ave_temperature']]
df.set_index('date', inplace=True)

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df.values)
window_size = 7

dataset = TimeSeriesDataset(scaled_data, window_size)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Model, loss, and optimizer
input_size = 1
hidden_size = 50
num_layers = 2
output_size = 7

model = LSTMModel(input_size, hidden_size, num_layers, output_size).float()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
num_epochs = 100
for epoch in range(num_epochs):
    for sequences, targets in train_loader:
        sequences = sequences.float()
        targets = targets.float()


        outputs = model(sequences)
        
        loss = criterion(outputs, targets.squeeze())  # targetsの次元を合わせる

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Predicting
model.eval()
with torch.no_grad():
    test_data = scaled_data[-window_size*2:]
    test_sequences = torch.tensor(test_data[:window_size]).float().unsqueeze(0)
    predictions = model(test_sequences)
    predictions = predictions.numpy().flatten()
    predictions = scaler.inverse_transform(predictions.reshape(-1, 1))

print("Predictions:", predictions)


torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size([16, 7, 1])
torch.Size(

KeyboardInterrupt: 